In [19]:
import numpy as np
import torch
import torch.backends.cudnn as cudnn
from utils.augmentations import letterbox
from models.common import DetectMultiBackend
from utils.general import (check_img_size,  check_requirements, non_max_suppression,  scale_coords, xyxy2xywh)
from utils.torch_utils import select_device
import cv2

import tracker


In [20]:
weights = 'yolov5s.pt'
data = 'data/coco128.yaml'
imgsz = (480,288)
conf_thres = 0.25
iou_thres=0.45
max_det=300
device_img=False
device=''

tracker = tracker.EuclideanDistTracker()

# Попробовать поменять dnn на тру, это может помочь ускорить обработку
dnn=True
source = 'http://192.168.66.55:8080/video'
half=False
# фильтр по классу(если нужно вывести что-то конкретное)
classes = 0
augment=False  # augmented inference
visualize=False
agnostic_nms=False


In [21]:
device = select_device(device)
model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
stride, names, pt = model.stride, model.names, model.pt
imgsz = check_img_size(imgsz, s=stride)  # check image size

YOLOv5 🚀 v6.1-313-g1c5e92a Python-3.8.10 torch-1.12.0+cu102 CUDA:0 (NVIDIA GeForce GTX 950, 1994MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients


In [22]:
def process_img(im0s):
    img = letterbox(im0s, imgsz, stride, auto=pt)[0]
    img = img.transpose((2, 0, 1))[::-1]
    img = np.ascontiguousarray(img)
    bs = 1  # batch_size
    model.warmup(imgsz=(1 if pt else bs, 3, *imgsz))  # warmup
    im = torch.from_numpy(img).to(device)
    im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
    im /= 255  # 0 - 255 to 0.0 - 1.0
    if len(im.shape) == 3:
        im = im[None]  # expand for batch dim
    pred = model(im, augment=augment, visualize=visualize)

    pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
    for i, det in enumerate(pred):  # per image
        if len(det):
            det[:, :4] = scale_coords(im.shape[2:], det[:, :4], im0s.shape).round()
            cropList = []
            for *xyxy, _, _ in reversed(det):
                xyxy = torch.tensor(xyxy).view(-1, 4)
                b = xyxy2xywh(xyxy)
                b = b.cpu().detach().numpy().tolist()
                cropList.append(b)
            return cropList

In [43]:
img = cv2.imread('image.jpeg')
# print(process_img(img))
cap = cv2.VideoCapture('street.mp4')
suc, img = cap.read()
bbxs = process_img(img)
# print(bbxs)
for id,i in enumerate(bbxs):
    x,y,w,h = int(i[0][0]),int(i[0][1]),int(i[0][2]),int(i[0][3])
    bbxs[id] = [x,y,w,h]
boxes_ids = tracker.update(bbxs)
for box_id in boxes_ids:
    x, y, w, h, id = box_id
    print(x,y,w,h)
    # cv2.putText(img, str(id), (int(x), int(y) - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
    cv2.rectangle(img, (int(x-w/2), int(y-h/2)), (int(x + w/2), int(y + h/2)), (0, 255, 0), -1)
# break    
show_frame = 1
out_frame = img
while suc:
    suc, img = cap.read()
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        # print(x,y,w,h)
        # cv2.putText(img, str(id), (int(x), int(y) - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
        cv2.rectangle(img, (int(x-w/2), int(y-h/2)), (int(x + w/2), int(y + h/2)), (0, 255, 0), -1)


    out = process_img(img)
    if(out):
        print('out',out)
        for id,i in enumerate(out):
            x,y,w,h = int(i[0][0]),int(i[0][1]),int(i[0][2]),int(i[0][3])
            # out[id] = [x,y,w,h]
            bbxs.append([x,y,w,h])
        boxes_ids = tracker.update(bbxs)
    else:
        show_frame = 0
    cv2.imshow('1', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break   
    

cv2.destroyAllWindows()

{6864: (130, 654), 6857: (313, 657), 6865: (1280, 720), 6866: (201, 659), 6867: (563, 512), 6868: (1001, 472), 6869: (1067, 696), 6870: (1135, 706), 6871: (742, 706), 6872: (663, 711), 6873: (484, 715), 6874: (1228, 429), 6875: (277, 480), 6876: (771, 481), 6877: (874, 444), 6878: (1197, 429), 6879: (243, 478), 6880: (1105, 453), 6881: (401, 454), 6882: (1245, 720), 6883: (689, 412), 6884: (814, 436), 6885: (1186, 720), 6886: (501, 427), 6887: (147, 418), 6888: (1256, 432), 6889: (816, 709), 6890: (990, 715), 6891: (994, 617), 6892: (873, 718), 6893: (932, 714), 6894: (845, 580), 6895: (1205, 616), 6896: (32, 689), 6897: (688, 484), 6898: (562, 717), 6899: (387, 685), 6900: (930, 487)}
{6864: (130, 654), 6857: (313, 657), 6865: (1280, 720), 6866: (201, 659), 6867: (563, 512), 6868: (1001, 472), 6869: (1067, 696), 6870: (1135, 706), 6871: (742, 706), 6872: (663, 711), 6873: (484, 715), 6874: (1228, 429), 6875: (277, 480), 6876: (771, 481), 6877: (874, 444), 6878: (1197, 429), 6879: (243